<a href="https://colab.research.google.com/github/annab0503/DS4002/blob/main/Project%201/Analysis%20Data/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Import dependencies
import pandas as pd
import numpy as np
import os
import requests

## Visa Data Frame

### Step 1: Load the Excel file from the URL

In [2]:
url = 'https://github.com/annab0503/DS4002/blob/main/Project%201/Original%20Data/FYs97-23_NIVDetailTable.xlsx?raw=true'
response = requests.get(url)
excel_file_path = '/content/FYs97-23_NIVDetailTable.xlsx'

# Save the file locally
with open(excel_file_path, 'wb') as f:
    f.write(response.content)

# Load the Excel file
excel_data = pd.ExcelFile(excel_file_path)

# Directory to save the CSV files
csv_directory = '/content/csv_files'
os.makedirs(csv_directory, exist_ok=True)  # Ensure directory exists

### Step 2: Iterate through each sheet in the Excel file and save as CSV

In [3]:
for sheet_name in excel_data.sheet_names:
    # Load the sheet into a DataFrame
    df = excel_data.parse(sheet_name)

    # Define the output CSV file name
    csv_file = os.path.join(csv_directory, f'{sheet_name}.csv')

    # Save the DataFrame as a CSV file
    df.to_csv(csv_file, index=False)
    print(f"Saved sheet '{sheet_name}' to {csv_file}")

Saved sheet 'FY97' to /content/csv_files/FY97.csv
Saved sheet 'FY98' to /content/csv_files/FY98.csv
Saved sheet 'FY99' to /content/csv_files/FY99.csv
Saved sheet 'FY00' to /content/csv_files/FY00.csv
Saved sheet 'FY01' to /content/csv_files/FY01.csv
Saved sheet 'FY02' to /content/csv_files/FY02.csv
Saved sheet 'FY03' to /content/csv_files/FY03.csv
Saved sheet 'FY04' to /content/csv_files/FY04.csv
Saved sheet 'FY05' to /content/csv_files/FY05.csv
Saved sheet 'FY06' to /content/csv_files/FY06.csv
Saved sheet 'FY07' to /content/csv_files/FY07.csv
Saved sheet 'FY08' to /content/csv_files/FY08.csv
Saved sheet 'FY09' to /content/csv_files/FY09.csv
Saved sheet 'FY10' to /content/csv_files/FY10.csv
Saved sheet 'FY11' to /content/csv_files/FY11.csv
Saved sheet 'FY12' to /content/csv_files/FY12.csv
Saved sheet 'FY13' to /content/csv_files/FY13.csv
Saved sheet 'FY14' to /content/csv_files/FY14.csv
Saved sheet 'FY15' to /content/csv_files/FY15.csv
Saved sheet 'FY16' to /content/csv_files/FY16.csv


### Step 3: Process each CSV file

In [4]:
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

# Loop over each CSV file for processing
for csv_file in csv_files:
    # Load the CSV file into a DataFrame
    year_data = pd.read_csv(os.path.join(csv_directory, csv_file))

    # Drop empty rows
    year_data = year_data.dropna(how='all')

    # Rename the 'Fiscal Year' column to 'Country'
    fiscal_year_column = next((col for col in year_data.columns if 'Fiscal Year' in col), None)
    if fiscal_year_column:
        year_data.rename(columns={fiscal_year_column: 'Country'}, inplace=True)

    # Remove rows for continents and totals
    countries_to_drop = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'Unknown']
    year_data = year_data[~year_data['Country'].isin(countries_to_drop)]
    year_data = year_data[~year_data['Country'].str.contains('Totals', na=False)]

    # Reshape the DataFrame with 'melt'
    year_data = year_data.melt(id_vars=['Country'],
                                var_name='Type of U.S. Visa',
                                value_name='Quantity of U.S. Visas Granted')

    # Extract year from the filename (e.g., 'FY1997.csv')
    year = int(csv_file.split('FY')[1].split('.csv')[0])
    year_data['Fiscal Year'] = year

    # Save the reshaped DataFrame as a new CSV file
    output_file = os.path.join(csv_directory, f"processed_{csv_file}")
    year_data.to_csv(output_file, index=False)

    print(f"Processed {csv_file} and saved to {output_file}")

Processed FY11.csv and saved to /content/csv_files/processed_FY11.csv
Processed FY09.csv and saved to /content/csv_files/processed_FY09.csv
Processed FY22.csv and saved to /content/csv_files/processed_FY22.csv


<ipython-input-4-b2c0cebc4b62>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_data.rename(columns={fiscal_year_column: 'Country'}, inplace=True)
<ipython-input-4-b2c0cebc4b62>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_data.rename(columns={fiscal_year_column: 'Country'}, inplace=True)


Processed FY08.csv and saved to /content/csv_files/processed_FY08.csv
Processed FY07.csv and saved to /content/csv_files/processed_FY07.csv
Processed FY04.csv and saved to /content/csv_files/processed_FY04.csv
Processed FY21.csv and saved to /content/csv_files/processed_FY21.csv
Processed FY15.csv and saved to /content/csv_files/processed_FY15.csv
Processed FY18.csv and saved to /content/csv_files/processed_FY18.csv
Processed FY13.csv and saved to /content/csv_files/processed_FY13.csv
Processed FY03.csv and saved to /content/csv_files/processed_FY03.csv
Processed FY99.csv and saved to /content/csv_files/processed_FY99.csv
Processed FY01.csv and saved to /content/csv_files/processed_FY01.csv
Processed FY16.csv and saved to /content/csv_files/processed_FY16.csv
Processed FY02.csv and saved to /content/csv_files/processed_FY02.csv
Processed FY10.csv and saved to /content/csv_files/processed_FY10.csv
Processed FY20.csv and saved to /content/csv_files/processed_FY20.csv
Processed FY00.csv a

### Step 4: Merge all processed CSV files into one DataFrame

In [5]:
processed_csv_files = [f for f in os.listdir(csv_directory) if f.startswith('processed_') and f.endswith('.csv')]

# List to hold DataFrames for merging
dfs = []

# Load each processed CSV file into a DataFrame and append to the list
for csv_file in processed_csv_files:
    df = pd.read_csv(os.path.join(csv_directory, csv_file))
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_csv_file = os.path.join(csv_directory, 'merged_data.csv')
merged_df.to_csv(merged_csv_file, index=False)

print(f"All processed CSV files merged and saved to {merged_csv_file}")

All processed CSV files merged and saved to /content/csv_files/merged_data.csv


 ### Step 5: Load the merged data

In [6]:
visa_data = pd.read_csv('/content/csv_files/merged_data.csv')

# Convert 'Year' column to numeric, coercing errors to NaN
visa_data['Fiscal Year'] = pd.to_numeric(visa_data['Fiscal Year'], errors='coerce')

### Step 6: Map fiscal year codes to actual years

In [7]:
code_to_year_map = {
    97: 1997, 98: 1998, 99: 1999, 0: 2000, 1: 2001, 2: 2002, 3: 2003, 4: 2004, 5: 2005,
    6: 2006, 7: 2007, 8: 2008, 9: 2009, 10: 2010, 11: 2011, 12: 2012, 13: 2013, 14: 2014,
    15: 2015, 16: 2016, 17: 2017, 18: 2018, 19: 2019, 20: 2020, 21: 2021, 22: 2022, 23: 2023
}

# Apply the mapping to the 'Year' column to convert codes to actual years
visa_data['Fiscal Year'] = visa_data['Fiscal Year'].map(code_to_year_map)

In [8]:
visa_data

,Country,Type of U.S. Visa,Quantity of U.S. Visas Granted,Fiscal Year
0,Algeria,A-1,26.0,2020
1,Angola,A-1,49.0,2020
2,Benin,A-1,7.0,2020
3,Botswana,A-1,6.0,2020
4,Burkina Faso,A-1,17.0,2020
...,...,...,...,...
466148,Suriname,Grand Total,1727.0,2004
466149,Uruguay,Grand Total,14676.0,2004
466150,Venezuela,Grand Total,79381.0,2004
466151,No Nationality,Grand Total,3744.0,2004


### Step 7: Export the merged and cleaned data frame

In [9]:
visa_data.to_csv('visa_data.csv', index=False)

## GDP Data Frame

### Step 1: Load the Excel file from the URL

In [10]:
url = 'https://github.com/annab0503/DS4002/blob/main/Project%201/Original%20Data/WITS-Country-Timeseries.xlsx?raw=true'
response = requests.get(url)
excel_file_path = '/WITS-Country-Timeseries.xlsx'

# Save the file locally
with open(excel_file_path, 'wb') as f:
    f.write(response.content)

# Load the Excel file
excel_data = pd.ExcelFile(excel_file_path)

# Load a specific sheet (e.g., the first sheet) into a DataFrame
df = excel_data.parse('Country-Timeseries')

# Display the first few rows of the DataFrame
df.head()

,Country Name,Indicator Name,1988,1989,1990,1991,1992,1993,1994,1995,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,GDP (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.056449e+10,2.055058e+10,1.999814e+10,1.801955e+10,1.889635e+10,1.841886e+10,1.890450e+10,2.014345e+10,1.458314e+10,NaN
1,Albania,GDP (current US$),2.051236e+09,2.253090e+09,2.028554e+09,1.099559e+09,6.521750e+08,1.185315e+09,1.880951e+09,2.392765e+09,...,1.277622e+10,1.322815e+10,1.138685e+10,1.186120e+10,1.301973e+10,1.515642e+10,1.540183e+10,1.516273e+10,1.793057e+10,1.888210e+10
2,Algeria,GDP (current US$),5.908940e+10,5.563472e+10,6.204851e+10,4.571568e+10,4.800313e+10,4.994559e+10,4.254318e+10,4.176429e+10,...,2.097550e+11,2.138100e+11,1.659792e+11,1.600342e+11,1.700970e+11,1.749109e+11,1.717603e+11,1.457437e+11,1.634722e+11,1.919129e+11
3,Andorra,GDP (current US$),7.214260e+08,7.954895e+08,1.028989e+09,1.106891e+09,1.209993e+09,1.007091e+09,1.017545e+09,1.178745e+09,...,3.193513e+09,3.271686e+09,2.789881e+09,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.325145e+09,3.352033e+09
4,Angola,GDP (current US$),8.769837e+09,1.020178e+10,1.122952e+10,1.270456e+10,1.511435e+10,1.105194e+10,3.390500e+09,5.561222e+09,...,1.334016e+11,1.372444e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.024137e+10,6.568544e+10,1.067136e+11


### Step 2: Process the data frame

In [11]:
df = df.melt(id_vars=["Country Name", "Indicator Name"],
                    var_name="Fiscal Year",
                    value_name="GDP")

df = df.drop(columns=["Indicator Name"])
df['Fiscal Year'] = pd.to_numeric(df['Fiscal Year'], errors='coerce')

df = df[df['Fiscal Year'].between(1997, 2014)]

df = df.dropna(subset=["GDP"])

# Rename 'Country Name' column to 'Country'
df.rename(columns={'Country Name': 'Country'}, inplace=True)

# Display the first few rows of the DataFrame
df

,Country,Fiscal Year,GDP
1738,Albania,1997,2.258514e+09
1739,Algeria,1997,4.817761e+10
1740,Andorra,1997,1.180646e+09
1741,Angola,1997,7.648377e+09
1742,Antigua and Barbuda,1997,6.806185e+08
...,...,...,...
5206,Venezuela,2014,4.823593e+11
5207,Vietnam,2014,2.334515e+11
5208,Yemen,2014,4.322859e+10
5209,Zambia,2014,2.714102e+10


### Step 3: Export the cleaned data frame

In [12]:
df = df.to_csv('/content/gdp_data.csv', index=False)

## Population Data

### Step 1: Load the Excel file from the URL

In [13]:
url = 'https://github.com/annab0503/DS4002/blob/main/Project%201/Original%20Data/IDB_01-03-2025.xlsx?raw=true'
response = requests.get(url)
excel_file_path = '/IDB_01-03-2025.xlsx'

# Save the file locally
with open(excel_file_path, 'wb') as f:
    f.write(response.content)

# Load the Excel file
df = pd.read_excel(excel_file_path)

# Display the first few rows of the DataFrame
df.head()

,Name,GENC,Year,Population,Annual Growth Rate,Rate of Natural Increase,Population Density,Crude Birth Rate,Crude Death Rate,Net Migration Rate,"Net International Migrants, Both Sexes"
0,-> 1997,NaN,NaN,--,--,--,--,--,--,--,--
1,Afghanistan,AF,1997.0,"21,514,488",3.76,3.77,33,50.6,12.9,-0.1,"-2,438"
2,Albania,AL,1997.0,"3,225,413",-0.01,1.35,117.7,19.5,6,-13.6,"-43,782"
3,Algeria,DZ,1997.0,"29,318,185",1.6,1.71,12.3,22.3,5.2,-1.2,"-33,826"
4,American Samoa,AS,1997.0,"55,918",--,--,282.4,--,--,--,--


### Step 2: Process the Data Frame

In [14]:
# Drop unnecessary rows and columns
df.dropna(subset=['Year'], inplace=True)
df = df.drop(columns=['GENC'])

# Define the columns to process
numeric_columns = [
    'Population',
    'Net International Migrants, Both Sexes',
    'Annual Growth Rate',
    'Rate of Natural Increase',
    'Population Density',
    'Crude Birth Rate',
    'Net Migration Rate'
]

# Process the columns
for col in numeric_columns:
    if col == 'Population' or col == 'Net International Migrants, Both Sexes':
        # For columns with commas, remove them, handle non-numeric values, and convert to integer
        df[col] = pd.to_numeric(df[col].str.replace(',', '', regex=False), errors='coerce').fillna(0).astype(int)
    else:
        # For other numeric columns, convert to float
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)

# Convert 'Year' column to integer
df['Year'] = df['Year'].astype(int)

# Rename the columns
df.rename(columns={'Name': 'Country'}, inplace=True)
df.rename(columns={'Year': 'Fiscal Year'}, inplace=True)

# Drop rows where 'Fiscal Year' = 2025
df = df[df['Fiscal Year'] != 2025]

df

,Country,Fiscal Year,Population,Annual Growth Rate,Rate of Natural Increase,Population Density,Crude Birth Rate,Crude Death Rate,Net Migration Rate,"Net International Migrants, Both Sexes"
1,Afghanistan,1997,21514488,3.76,3.77,33.0,50.6,12.9,-0.1,-2438
2,Albania,1997,3225413,-0.01,1.35,117.7,19.5,6,-13.6,-43782
3,Algeria,1997,29318185,1.60,1.71,12.3,22.3,5.2,-1.2,-33826
4,American Samoa,1997,55918,0.00,0.00,282.4,0.0,--,0.0,0
5,Andorra,1997,64053,1.23,0.67,136.9,11.4,4.7,5.7,0
...,...,...,...,...,...,...,...,...,...,...
6151,Wallis and Futuna,2023,15929,0.23,0.62,112.2,12.1,5.9,-3.8,0
6152,West Bank,2023,3176549,2.10,2.48,563.2,28.3,3.5,-3.9,-12271
6153,Yemen,2023,32968634,2.34,2.40,62.4,30.2,6.2,-0.7,-22423
6154,Zambia,2023,20216029,2.86,2.85,27.2,34.5,6,0.2,3144


In [15]:
# Create a new row for Palestine by grouping Gaza Strip and West Bank
palestine_row = df[df['Country'].isin(['Gaza Strip', 'West Bank'])]

# Define the columns to sum and average
sum_columns = ['Population']
avg_columns = [
    'Annual Growth Rate',
    'Rate of Natural Increase',
    'Population Density',
    'Crude Birth Rate',
    'Net Migration Rate',
    'Net International Migrants, Both Sexes'
]

# Sum the columns that need to be summed
palestine_row_sum = palestine_row.groupby('Fiscal Year')[sum_columns].sum()

# Average the columns that need to be averaged
palestine_row_avg = palestine_row.groupby('Fiscal Year')[avg_columns].mean()

# Combine the sum and average results into one DataFrame
palestine_row = pd.concat([palestine_row_sum, palestine_row_avg], axis=1)

# Add 'Country' as 'Palestine' to the new row
palestine_row['Country'] = 'Palestine'

# Reset the index so it's a DataFrame
palestine_row = palestine_row.reset_index()

# Concatenate the Palestine row to the DataFrame
df = pd.concat([df, palestine_row], ignore_index=True)

df

,Country,Fiscal Year,Population,Annual Growth Rate,Rate of Natural Increase,Population Density,Crude Birth Rate,Crude Death Rate,Net Migration Rate,"Net International Migrants, Both Sexes"
0,Afghanistan,1997,21514488,3.760,3.770,33.00,50.60,12.9,-0.10,-2438.0
1,Albania,1997,3225413,-0.010,1.350,117.70,19.50,6,-13.60,-43782.0
2,Algeria,1997,29318185,1.600,1.710,12.30,22.30,5.2,-1.20,-33826.0
3,American Samoa,1997,55918,0.000,0.000,282.40,0.00,--,0.00,0.0
4,Andorra,1997,64053,1.230,0.670,136.90,11.40,4.7,5.70,0.0
...,...,...,...,...,...,...,...,...,...,...
6151,Palestine,2019,4853985,1.970,2.645,258.75,29.70,NaN,-6.75,-15135.5
6152,Palestine,2020,4955891,2.120,2.530,264.55,29.05,NaN,-4.10,-10135.5
6153,Palestine,2021,5059569,2.015,2.415,270.10,29.00,NaN,-4.00,-10135.5
6154,Palestine,2022,5165249,2.115,2.510,275.75,28.50,NaN,-3.95,-10135.5


### Step 3: Export the cleaned data frame

In [16]:
df = df.to_csv('/content/pop_data.csv', index=False)

## Merged Data Frame

### Step 1: Load the cleaned data frames

In [17]:
# Load the GDP data and visa data
gdp_data = pd.read_csv('gdp_data.csv')
visa_data = pd.read_csv('visa_data.csv')
pop_data = pd.read_csv('pop_data.csv')

# Ensure the columns 'Country' and 'Year' are present and consistent in both DataFrames
print("Columns in GDP Data:", gdp_data.columns)
print("Columns in Visa Data:", visa_data.columns)
print("Columns in Population Data:", pop_data.columns)

# Standardize column names if necessary (e.g., trimming whitespaces or renaming)
gdp_data.rename(columns=lambda x: x.strip(), inplace=True)
visa_data.rename(columns=lambda x: x.strip(), inplace=True)
pop_data.rename(columns=lambda x: x.strip(), inplace=True)

Columns in GDP Data: Index(['Country', 'Fiscal Year', 'GDP'], dtype='object')
Columns in Visa Data: Index(['Country', 'Type of U.S. Visa', 'Quantity of U.S. Visas Granted',
       'Fiscal Year'],
      dtype='object')
Columns in Population Data: Index(['Country', 'Fiscal Year', 'Population', 'Annual Growth Rate',
       'Rate of Natural Increase', 'Population Density', 'Crude Birth Rate',
       'Crude Death Rate', 'Net Migration Rate',
       'Net International Migrants, Both Sexes'],
      dtype='object')


### Step 2: Troubleshoot unmatched data

In [18]:
# Identify unique values of 'Country' in the unmatched_data and sort alphabetically
unique_countries1 = sorted(visa_data['Country'].unique())
unique_countries2 = sorted(gdp_data['Country'].unique())
unique_countries3 = sorted(pop_data['Country'].unique())

In [19]:
# Convert the lists of unique countries into sets
set_countries1 = set(unique_countries1)
set_countries2 = set(unique_countries2)
set_countries3 = set(unique_countries3)

In [20]:
missing_in_visa = set_countries1 - set_countries2 - set_countries3
missing_in_visa

{'Bosnia-Herzegovina',
 'China - Taiwan',
 'China - mainland',
 'Congo, Dem. Rep. of the (Congo Kinshasa)',
 'Congo, Dem. Rep. of the (Kinshasa)',
 'Congo, Democratic Republic of the',
 'Congo, Democratic Republic of the ',
 'Congo, Rep. of the (Brazzaville)',
 'Congo, Rep. of the (Congo Brazzaville)',
 'Congo, Republic of the',
 'Congo, Republic of the ',
 "Cote d'Ivoire ",
 'Grand Total',
 'Great Britain and Northern Ireland',
 'Hong Kong S.A.R.',
 'Macau S.A.R.',
 'Macedonia',
 'Micronesia',
 'No Nationality',
 'Palestinian Authority Travel Document',
 'Serbia  ',
 'Serbia and Montenegro',
 'Swaziland',
 'United Nations Laissez-Passer',
 'Vatican City'}

In [21]:
# Rename Countries in Visa data frame for consistency
visa_data.loc[visa_data['Country'] == 'Bosnia-Herzegovina', 'Country'] = 'Bosnia and Herzegovina'
visa_data.loc[visa_data['Country'] == 'China - Taiwan', 'Country'] = 'Taiwan'
visa_data.loc[visa_data['Country'] == 'China - mainland', 'Country'] = 'China'
visa_data.loc[visa_data['Country'] == 'Congo, Dem. Rep. of the (Congo Kinshasa)', 'Country'] = 'Democratic Republic of the Congo'
visa_data.loc[visa_data['Country'] == 'Congo, Dem. Rep. of the (Kinshasa)', 'Country'] = 'Democratic Republic of the Congo'
visa_data.loc[visa_data['Country'] == 'Congo, Democratic Republic of the', 'Country'] = 'Democratic Republic of the Congo'
visa_data.loc[visa_data['Country'] == 'Congo, Rep. of the (Congo Brazzaville)', 'Country'] = 'Republic of the Congo'
visa_data.loc[visa_data['Country'] == 'Congo, Rep. of the (Brazzaville)', 'Country'] = 'Republic of the Congo'
visa_data.loc[visa_data['Country'] == 'Congo, Republic of the', 'Country'] = 'Republic of the Congo'
visa_data.loc[visa_data['Country'] == 'Congo, Democratic Republic of the ', 'Country'] = 'Republic of the Congo'
visa_data.loc[visa_data['Country'] == "Cote d'Ivoire ", 'Country'] = "Cote d'Ivoire"
visa_data.loc[visa_data['Country'] == 'Great Britain and Northern Ireland', 'Country'] = 'United Kingdom'
visa_data.loc[visa_data['Country'] == 'Hong Kong S.A.R.', 'Country'] = 'Hong Kong'
visa_data.loc[visa_data['Country'] == 'Macau S.A.R.', 'Country'] = 'Macau'
visa_data.loc[visa_data['Country'] == 'Kyrgyzstan', 'Country'] = 'Kyrgyz Republic'
visa_data.loc[visa_data['Country'] == 'Macedonia', 'Country'] = 'North Macedonia'
visa_data.loc[visa_data['Country'] == 'Palestinian Authority Travel Document', 'Country'] = 'Palestine'
visa_data.loc[visa_data['Country'] == 'Serbia  ', 'Country'] = 'Serbia'
visa_data.loc[visa_data['Country'] == 'Swaziland', 'Country'] = 'Eswatini (Swaziland)'
visa_data.loc[visa_data['Country'] == 'Eswatini', 'Country'] = 'Eswatini (Swaziland)'

In [22]:
missing_in_gdp = set_countries2 - set_countries1 - set_countries3
missing_in_gdp

{'Congo, Dem. Rep.',
 'Congo, Rep.',
 'East Timor',
 'Egypt, Arab Rep.',
 'Ethiopia(excludes Eritrea)',
 'Faeroe Islands',
 'Fm Sudan',
 'Hong Kong, China',
 'Iran, Islamic Rep.',
 'Korea, Rep.',
 'Kyrgyz Republic',
 'Lao PDR',
 'Macao',
 'Micronesia, Fed. Sts.',
 'Myanmar',
 'Occ.Pal.Terr',
 'Russian Federation',
 'Serbia, FR(Serbia/Montenegro)',
 'Slovak Republic',
 'St. Kitts and Nevis',
 'St. Lucia',
 'St. Vincent and the Grenadines',
 'Syrian Arab Republic',
 'Turks and Caicos Isl.'}

In [23]:
# Rename Countries in GDP data frame for consistency
gdp_data.loc[gdp_data['Country'] == 'Congo, Dem. Rep.', 'Country'] = 'Democratic Republic of the Congo'
gdp_data.loc[gdp_data['Country'] == 'Congo, Rep.', 'Country'] = 'Republic of the Congo'
gdp_data.loc[gdp_data['Country'] == 'East Timor', 'Country'] = 'Timor-Leste'
gdp_data.loc[gdp_data['Country'] == 'Egypt, Arab Rep.', 'Country'] = 'Egypt'
gdp_data.loc[gdp_data['Country'] == 'Ethiopia(excludes Eritrea)', 'Country'] = 'Ethiopia'
gdp_data.loc[gdp_data['Country'] == 'Fm Sudan', 'Country'] = 'Sudan'
gdp_data.loc[gdp_data['Country'] == 'Hong Kong, China', 'Country'] = 'Hong Kong'
gdp_data.loc[gdp_data['Country'] == 'Iran, Islamic Rep.', 'Country'] = 'Iran'
gdp_data.loc[gdp_data['Country'] == 'Korea, Rep.', 'Country'] = 'Korea, South'
gdp_data.loc[gdp_data['Country'] == 'Lao PDR', 'Country'] = 'Laos'
gdp_data.loc[gdp_data['Country'] == 'Macao', 'Country'] = 'Macau'
gdp_data.loc[gdp_data['Country'] == 'Micronesia, Fed. Sts.', 'Country'] = 'Micronesia'
gdp_data.loc[gdp_data['Country'] == 'Myanmar', 'Country'] = 'Burma'
gdp_data.loc[gdp_data['Country'] == 'Myanmar', 'Country'] = 'Palestine'
gdp_data.loc[gdp_data['Country'] == 'Russian Federation', 'Country'] = 'Russia'
gdp_data.loc[gdp_data['Country']== 'Serbia, FR(Serbia/Montenegro)', 'Country'] = 'Serbia and Montenegro'
gdp_data.loc[gdp_data['Country'] == 'Slovak Republic', 'Country'] = 'Slovakia'
gdp_data.loc[gdp_data['Country'] == 'St. Kitts and Nevis', 'Country'] = 'Saint Kitts and Nevis'
gdp_data.loc[gdp_data['Country']== 'St. Lucia', 'Country'] = 'Saint Lucia'
gdp_data.loc[gdp_data['Country'] == 'St. Vincent and the Grenadines', 'Country'] = 'Saint Vincent and the Grenadines'
gdp_data.loc[gdp_data['Country'] == 'Syrian Arab Republic', 'Country'] = 'Syria'

In [24]:
missing_in_pop = set_countries3 - set_countries1 - set_countries2
missing_in_pop

{'American Samoa',
 'Anguilla',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Cook Islands',
 'Curaçao',
 'Czechia',
 'Côte d’Ivoire',
 'Faroe Islands',
 'Gaza Strip',
 'Gibraltar',
 'Guam',
 'Guernsey',
 'Hong Kong',
 'Isle of Man',
 'Jersey',
 'Macau',
 'Montserrat',
 'Northern Mariana Islands',
 'Palestine',
 'Puerto Rico',
 'Saint Barthelemy',
 'Saint Helena, Ascension, and Tristan da Cunha',
 'Saint Martin',
 'Saint Pierre and Miquelon',
 'Sint Maarten',
 'Turks and Caicos Islands',
 'Virgin Islands, British',
 'Virgin Islands, U.S.',
 'Wallis and Futuna',
 'West Bank'}

In [25]:
# Rename Countries in Population data frame for consistency
pop_data.loc[pop_data['Country'] == 'Congo (Brazzaville)', 'Country'] = 'Republic of the Congo'
pop_data.loc[pop_data['Country'] == 'Congo (Kinshasa)', 'Country'] = 'Democratic Republic of the Congo'
pop_data.loc[pop_data['Country'] == 'Czechia', 'Country'] = 'Czech Republic'
pop_data.loc[pop_data['Country'] == "Côte d’Ivoire", 'Country'] = "Cote d'Ivoire"

### Step 3: Complete merge

In [26]:
# Merge the data frames on 'Country' and 'Fiscal Year' with indicator=True
merged_data = pd.merge(visa_data, gdp_data, on=['Country', 'Fiscal Year'], how='outer', indicator=True)

# Rename the '_merge' column to avoid conflict
merged_data.rename(columns={'_merge': '_merge_gdp'}, inplace=True)

# Merge with the pop_data DataFrame
merged_data = pd.merge(merged_data, pop_data, on=['Country', 'Fiscal Year'], how='outer', indicator=True)

### Step 4: Process the data frame

In [27]:
merged_data = merged_data.drop(columns=['_merge', '_merge_gdp'])

# Convert the specified columns to integers
columns_to_convert = [
    "Quantity of U.S. Visas Granted",
    "Population",
    "Net International Migrants, Both Sexes"
]

# Apply the conversion
for col in columns_to_convert:
    merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce').fillna(0).astype(int)

# Drop rows where 'Type of U.S. Visa' is NaN
merged_data = merged_data.dropna(subset=['Type of U.S. Visa'])

# Strip dashes from 'Type of U.S. Visa'
merged_data['Type of U.S. Visa'] = merged_data['Type of U.S. Visa'].str.replace('-', '', regex=False)

<ipython-input-27-8a5ab77fed47>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['Type of U.S. Visa'] = merged_data['Type of U.S. Visa'].str.replace('-', '', regex=False)


In [28]:
unique_visas = sorted(merged_data['Type of U.S. Visa'].unique())
unique_visas

['A1',
 'A2',
 'A3',
 'B1',
 'B1,2',
 'B1,2/BCC',
 'B1,2/BCV',
 'B1/B2',
 'B2',
 'BBBCC',
 'BBBCV',
 'BCC',
 'C1',
 'C1/D',
 'C2',
 'C3',
 'CW1',
 'CW2',
 'D',
 'DCREW',
 'E1',
 'E2',
 'E2C',
 'E3',
 'E3D',
 'E3R',
 'F1',
 'F2',
 'F3',
 'G1',
 'G2',
 'G3',
 'G4',
 'G5',
 'Grand Total',
 'H1A',
 'H1B',
 'H1B1',
 'H1C',
 'H2A',
 'H2B',
 'H2R',
 'H3',
 'H4',
 'I',
 'J1',
 'J2',
 'K1',
 'K2',
 'K3',
 'K4',
 'L1',
 'L2',
 'M1',
 'M2',
 'M3',
 'N8',
 'N9',
 'NATO1',
 'NATO2',
 'NATO3',
 'NATO4',
 'NATO5',
 'NATO6',
 'NATO7',
 'O1',
 'O2',
 'O3',
 'P1',
 'P2',
 'P3',
 'P4',
 'Q1',
 'Q2',
 'Q3',
 'R1',
 'R2',
 'S5',
 'S6',
 'S7',
 'T1',
 'T2',
 'T3',
 'T4',
 'T5',
 'T6',
 'TD',
 'TN',
 'Total Visas',
 'U1',
 'U2',
 'U3',
 'U4',
 'U5',
 'V1',
 'V2',
 'V3']

In [29]:
# Define a mapping for the visa categories
visa_category_map = {
    # Diplomatic and Official Visas
    'A1': 'Diplomatic and Official Visas', 'A2': 'Diplomatic and Official Visas', 'A3': 'Diplomatic and Official Visas',
    'G1': 'Diplomatic and Official Visas', 'G2': 'Diplomatic and Official Visas', 'G3': 'Diplomatic and Official Visas',
    'G4': 'Diplomatic and Official Visas', 'G5': 'Diplomatic and Official Visas',
    'NATO1': 'Diplomatic and Official Visas', 'NATO2': 'Diplomatic and Official Visas', 'NATO3': 'Diplomatic and Official Visas',
    'NATO4': 'Diplomatic and Official Visas', 'NATO5': 'Diplomatic and Official Visas', 'NATO6': 'Diplomatic and Official Visas',
    'NATO7': 'Diplomatic and Official Visas',
    # Visitor Visas
    'B1': 'Visitor Visas', 'B1,2': 'Visitor Visas', 'B1,2/BCC': 'Visitor Visas', 'B1,2/BCV': 'Visitor Visas', 'B1/B2': 'Visitor Visas',
    'B2': 'Visitor Visas',
    # Student and Exchange Visitor Visas
    'F1': 'Student and Exchange Visitor Visas', 'F2': 'Student and Exchange Visitor Visas', 'F3': 'Student and Exchange Visitor Visas',
    'J1': 'Student and Exchange Visitor Visas', 'J2': 'Student and Exchange Visitor Visas',
    'M1': 'Student and Exchange Visitor Visas', 'M2': 'Student and Exchange Visitor Visas',
    # Skilled Labor Visas
    'H1B': 'Skilled Labor Visas', 'H1B1': 'Skilled Labor Visas', 'L1': 'Skilled Labor Visas', 'O1': 'Skilled Labor Visas',
    'O2': 'Skilled Labor Visas', 'P1': 'Skilled Labor Visas', 'P2': 'Skilled Labor Visas', 'P3': 'Skilled Labor Visas',
    'Q1': 'Skilled Labor Visas', 'R1': 'Skilled Labor Visas',
    # Unskilled Labor Visas
    'H2A': 'Unskilled Labor Visas', 'H2B': 'Unskilled Labor Visas', 'H2R': 'Unskilled Labor Visas', 'H3': 'Unskilled Labor Visas',
    'O3': 'Unskilled Labor Visas', 'P4': 'Unskilled Labor Visas', 'R2': 'Unskilled Labor Visas',
    # Immigrant and Family-Based Visas
    'K1': 'Immigrant and Family-Based Visas', 'K2': 'Immigrant and Family-Based Visas', 'K3': 'Immigrant and Family-Based Visas',
    'K4': 'Immigrant and Family-Based Visas', 'U1': 'Immigrant and Family-Based Visas', 'U2': 'Immigrant and Family-Based Visas',
    'U3': 'Immigrant and Family-Based Visas', 'U4': 'Immigrant and Family-Based Visas', 'U5': 'Immigrant and Family-Based Visas',
    'V1': 'Immigrant and Family-Based Visas', 'V2': 'Immigrant and Family-Based Visas', 'V3': 'Immigrant and Family-Based Visas',
    # Temporary Worker Visas
    'C1': 'Temporary Worker Visas', 'C1/D': 'Temporary Worker Visas', 'D': 'Temporary Worker Visas', 'D-CREW': 'Temporary Worker Visas',
    'T1': 'Temporary Worker Visas', 'T2': 'Temporary Worker Visas', 'T3': 'Temporary Worker Visas', 'T4': 'Temporary Worker Visas',
    'T5': 'Temporary Worker Visas', 'T6': 'Temporary Worker Visas', 'S5': 'Temporary Worker Visas', 'S6': 'Temporary Worker Visas',
    'S7': 'Temporary Worker Visas', 'TN': 'Temporary Worker Visas',
    # Media Visas
    'I': 'Media Visas',
    # Total Visas
    'Total Visas': 'Total',
    # Grand Total
    'Grand Total': 'Grand Total'
}

# Apply the categorization using the correct map
merged_data['Visa Category'] = merged_data['Type of U.S. Visa'].map(visa_category_map).fillna('Uncategorized')

In [30]:
# Define the desired column order
column_order = [
    'Country', 'Fiscal Year', 'Type of U.S. Visa', 'Visa Category', 'Quantity of U.S. Visas Granted',
    'Population', 'GDP', 'Annual Growth Rate', 'Rate of Natural Increase',
    'Population Density', 'Crude Birth Rate', 'Net Migration Rate', 'Net International Migrants, Both Sexes'
]

# Reorganize the columns
merged_data = merged_data[column_order]

In [31]:
merged_data

,Country,Fiscal Year,Type of U.S. Visa,Visa Category,Quantity of U.S. Visas Granted,Population,GDP,Annual Growth Rate,Rate of Natural Increase,Population Density,Crude Birth Rate,Net Migration Rate,"Net International Migrants, Both Sexes"
0,Afghanistan,1997,A1,Diplomatic and Official Visas,0,21514488,NaN,3.76,3.77,33.0,50.6,-0.1,-2438
1,Afghanistan,1997,A2,Diplomatic and Official Visas,1,21514488,NaN,3.76,3.77,33.0,50.6,-0.1,-2438
2,Afghanistan,1997,A3,Diplomatic and Official Visas,1,21514488,NaN,3.76,3.77,33.0,50.6,-0.1,-2438
3,Afghanistan,1997,B1,Visitor Visas,46,21514488,NaN,3.76,3.77,33.0,50.6,-0.1,-2438
4,Afghanistan,1997,"B1,2",Visitor Visas,361,21514488,NaN,3.76,3.77,33.0,50.6,-0.1,-2438
...,...,...,...,...,...,...,...,...,...,...,...,...,...
467181,Zimbabwe,2023,U2,Immigrant and Family-Based Visas,0,16819805,NaN,1.99,2.27,43.5,29.4,-2.9,-47935
467182,Zimbabwe,2023,U3,Immigrant and Family-Based Visas,0,16819805,NaN,1.99,2.27,43.5,29.4,-2.9,-47935
467183,Zimbabwe,2023,U4,Immigrant and Family-Based Visas,0,16819805,NaN,1.99,2.27,43.5,29.4,-2.9,-47935
467184,Zimbabwe,2023,U5,Immigrant and Family-Based Visas,0,16819805,NaN,1.99,2.27,43.5,29.4,-2.9,-47935


### Step 5: Export the merged data frame

In [32]:
merged_data.to_csv('merged_data.csv', index=False)

In [39]:
# Define the number of rows per CSV file (split into two files)
rows_per_file = len(merged_data) // 2

# Split the DataFrame into two parts for upload
for i in range(2):
    chunk = merged_data.iloc[i * rows_per_file:(i + 1) * rows_per_file] if i < 1 else merged_data.iloc[i * rows_per_file:]
    chunk.to_csv(f'merged_data_{i + 1}.csv', index=False)
    print(f"Saved chunk {i + 1} as 'merged_data_{i + 1}.csv'")


Saved chunk 1 as 'merged_data_1.csv'
Saved chunk 2 as 'merged_data_2.csv'


## Analysis Data Frame

In [45]:
# Drop 'Type of U.S. Visa
analysis_data = merged_data.drop(columns=['Type of U.S. Visa'])

# Drop rows with 'Fiscal Year' prior to 2022
analysis_data = analysis_data[analysis_data['Fiscal Year'] <= 2022]

# Group by all columns except "Quantity of U.S. Visas Granted"
columns_to_group = [col for col in analysis_data.columns if col != "Quantity of U.S. Visas Granted"]

# Group the DataFrame and sum "Quantity of U.S. Visas Granted"
analysis_data = analysis_data.groupby(columns_to_group, as_index=False).agg({
    "Quantity of U.S. Visas Granted": "sum",  # Sum the visa quantity
    **{col: "first" for col in columns_to_group if col != "Quantity of U.S. Visas Granted"}  # Keep the first value for other columns
})

# redefine the desired column order
column_order = [
    'Country', 'Fiscal Year','Visa Category', 'Quantity of U.S. Visas Granted',
    'Population', 'GDP', 'Annual Growth Rate', 'Rate of Natural Increase',
    'Population Density', 'Crude Birth Rate', 'Net Migration Rate', 'Net International Migrants, Both Sexes'
]

# Reorganize the columns
analysis_data = analysis_data[column_order]

# Display the result
analysis_data

,Country,Fiscal Year,Visa Category,Quantity of U.S. Visas Granted,Population,GDP,Annual Growth Rate,Rate of Natural Increase,Population Density,Crude Birth Rate,Net Migration Rate,"Net International Migrants, Both Sexes"
0,Afghanistan,2002,Diplomatic and Official Visas,181,25121599,3.854235e+09,8.15,2.97,38.5,40.2,51.7,1299482
1,Afghanistan,2002,Grand Total,1121,25121599,3.854235e+09,8.15,2.97,38.5,40.2,51.7,1299482
2,Afghanistan,2002,Immigrant and Family-Based Visas,205,25121599,3.854235e+09,8.15,2.97,38.5,40.2,51.7,1299482
3,Afghanistan,2002,Media Visas,0,25121599,3.854235e+09,8.15,2.97,38.5,40.2,51.7,1299482
4,Afghanistan,2002,Skilled Labor Visas,9,25121599,3.854235e+09,8.15,2.97,38.5,40.2,51.7,1299482
...,...,...,...,...,...,...,...,...,...,...,...,...
34887,Zimbabwe,2014,Temporary Worker Visas,467,13791770,1.949552e+10,2.08,2.63,35.7,35.4,-5.5,-75299
34888,Zimbabwe,2014,Total,7358,13791770,1.949552e+10,2.08,2.63,35.7,35.4,-5.5,-75299
34889,Zimbabwe,2014,Uncategorized,37,13791770,1.949552e+10,2.08,2.63,35.7,35.4,-5.5,-75299
34890,Zimbabwe,2014,Unskilled Labor Visas,7,13791770,1.949552e+10,2.08,2.63,35.7,35.4,-5.5,-75299


In [46]:
# Filter the rows to keep only the relevant visa categories
analysis_data = analysis_data[analysis_data['Visa Category'].isin(['Skilled Labor Visas', 'Unskilled Labor Visas', 'Student and Exchange Visitor Visas'])]

# Drop rows where GDP is NaN
analysis_data = analysis_data.dropna(subset=['GDP'])

# Display the updated DataFrame
analysis_data

,Country,Fiscal Year,Visa Category,Quantity of U.S. Visas Granted,Population,GDP,Annual Growth Rate,Rate of Natural Increase,Population Density,Crude Birth Rate,Net Migration Rate,"Net International Migrants, Both Sexes"
4,Afghanistan,2002,Skilled Labor Visas,9,25121599,3.854235e+09,8.15,2.97,38.5,40.2,51.7,1299482
5,Afghanistan,2002,Student and Exchange Visitor Visas,38,25121599,3.854235e+09,8.15,2.97,38.5,40.2,51.7,1299482
9,Afghanistan,2002,Unskilled Labor Visas,0,25121599,3.854235e+09,8.15,2.97,38.5,40.2,51.7,1299482
15,Afghanistan,2003,Skilled Labor Visas,9,26829863,4.539497e+09,5.11,3.71,41.1,47.3,14.0,374242
16,Afghanistan,2003,Student and Exchange Visitor Visas,101,26829863,4.539497e+09,5.11,3.71,41.1,47.3,14.0,374242
...,...,...,...,...,...,...,...,...,...,...,...,...
34875,Zimbabwe,2013,Student and Exchange Visitor Visas,681,13504275,1.909102e+10,2.13,2.63,34.9,35.7,-5.0,-67451
34879,Zimbabwe,2013,Unskilled Labor Visas,8,13504275,1.909102e+10,2.13,2.63,34.9,35.7,-5.0,-67451
34885,Zimbabwe,2014,Skilled Labor Visas,179,13791770,1.949552e+10,2.08,2.63,35.7,35.4,-5.5,-75299
34886,Zimbabwe,2014,Student and Exchange Visitor Visas,763,13791770,1.949552e+10,2.08,2.63,35.7,35.4,-5.5,-75299


In [48]:
analysis_data.to_csv('analysis_data.csv', index=False)